In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [2]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/04/09 18:49:06] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/09 18:49:06] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/04/09 18:49:06] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [2]:
gui1 = GUI('data/UIComparison/80.png')
# gui1.detect_element(True, True, True, paddle_cor=paddle_ocr)
gui1.load_detection_result()
gui1.show_detection_result()

In [5]:
gui2 = GUI('data/UIComparison/60.png')
gui2.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui1.load_detection_result()
gui2.show_detection_result()

[2022/04/09 18:50:06] root DEBUG: dt_boxes num : 32, elapse : 0.1388866901397705
[2022/04/09 18:50:06] root DEBUG: cls num  : 32, elapse : 0.11372685432434082
[2022/04/09 18:50:07] root DEBUG: rec_res num  : 32, elapse : 0.19258332252502441
[Text Detection Completed in 0.505 s] Input: data/UIComparison/60.png Output: data/output/ocr/60.json
[Compo Detection Completed in 0.450 s] Input: data/UIComparison/60.png Output: data/output/ip/60.json
[Merge Completed] Input: data/UIComparison/60.png Output: data/output/merge/60.jpg


In [6]:
gui3 = GUI('data/UIComparison/50.png')
gui3.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui1.load_detection_result()
gui3.show_detection_result()

[2022/04/09 18:50:24] root DEBUG: dt_boxes num : 29, elapse : 0.16683387756347656
[2022/04/09 18:50:24] root DEBUG: cls num  : 29, elapse : 0.0996859073638916
[2022/04/09 18:50:25] root DEBUG: rec_res num  : 29, elapse : 0.1550447940826416
[Text Detection Completed in 0.475 s] Input: data/UIComparison/50.png Output: data/output/ocr/50.json
[Compo Detection Completed in 0.519 s] Input: data/UIComparison/50.png Output: data/output/ip/50.json
[Merge Completed] Input: data/UIComparison/50.png Output: data/output/merge/50.jpg


In [7]:
gui4 = GUI('data/UIComparison/37.png')
gui4.detect_element(True, True, True, paddle_cor=paddle_ocr)
# gui1.load_detection_result()
gui4.show_detection_result()

[2022/04/09 18:50:38] root DEBUG: dt_boxes num : 20, elapse : 0.14643526077270508
[2022/04/09 18:50:38] root DEBUG: cls num  : 20, elapse : 0.08402824401855469
[2022/04/09 18:50:38] root DEBUG: rec_res num  : 20, elapse : 0.12517690658569336
[Text Detection Completed in 0.371 s] Input: data/UIComparison/37.png Output: data/output/ocr/37.json
[Compo Detection Completed in 0.435 s] Input: data/UIComparison/37.png Output: data/output/ip/37.json
[Merge Completed] Input: data/UIComparison/37.png Output: data/output/merge/37.jpg


In [3]:
e = gui1.get_element_by_clicking()

Click (211, 542)


In [4]:
e

(None, -1, -1)